<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Введение" data-toc-modified-id="Введение-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Введение</a></span></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Подготовки-библиотек-и-функций" data-toc-modified-id="Подготовки-библиотек-и-функций-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Подготовки библиотек и функций</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Датасет-для-обучения" data-toc-modified-id="Датасет-для-обучения-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Датасет для обучения</a></span></li><li><span><a href="#Отложенные-датасет" data-toc-modified-id="Отложенные-датасет-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Отложенные датасет</a></span></li></ul></li><li><span><a href="#Получение-эмбеддингов" data-toc-modified-id="Получение-эмбеддингов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Получение эмбеддингов</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Оценка-качества-модели" data-toc-modified-id="Оценка-качества-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Оценка качества модели</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Введение
**Цель работы**: разработать модель, способную классифицировать твиты на токсичные и нетоксичные. Полученная модель должна иметь метрику качества F1 не ниже 0.75.

**Задачи, решаемые в работе:**
1. Подготовка данных для прогнозирования.
2. Обучение нескольких моделей и выбор лучшей.
3. Оценка качества работы модели.

**План работы**:
1. Загрузка данных и разбиение их на выборки.
2. Очистка и подготовка данных.
3. Получение эмбреддингов.
4. Формирование pipeline для обучения нескольких моделей.
5. Оценка качества лучшей модели.

**Описание работы:**
Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.


## Подготовка

### Подготовки библиотек и функций

Сперва загрузим нужные модули:

In [ ]:
!pip install pymystem3 -q
!pip install torch -q
!pip install lightgbm -q
!pip install catboost -q
!pip install xgboost -q
!pip install Pillow==8.0.0 -q
!pip install wordcloud -q

Теперь загрузим нужные библиотеки

In [ ]:
#Общие библиотеки
import numpy as np
import pandas as pd
import os
import torch
import transformers
from tqdm import notebook

# Библиотеки для работы с моделями
from catboost import CatBoostClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (train_test_split,
                                     GridSearchCV,
                                     StratifiedShuffleSplit,
                                     StratifiedKFold)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from transformers import BertTokenizer, BertModel

# Библиотеки для работы с текстом
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from pymystem3 import Mystem
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud

# Библиотеки для оценки качества
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score

# Библиотеки для графиков
import matplotlib.pyplot as plt
import seaborn as sns

Заранее подготовим необходимые функции

In [ ]:
def is_running_in_colab():
    """Проверяет, выполняется ли код в Google Colab"""
    try:
        import google.colab
        return True
    except ImportError:
        return False

In [ ]:
def load_dataframe():
    # Попытка загрузить локальный файл
    try:
        df = pd.read_csv('F:\\Загрузки\\toxic_comments.csv')
        print("Файл успешно загружен из локального источника")
        return df
    except FileNotFoundError:
        print("Локальный файл не найден, пробую загрузить с платформы...")
    except Exception as e:
        print(f"Ошибка при загрузке локального файла: {str(e)}")

    # Попытка загрузить файл с платформы
    try:
        df = pd.read_csv('/datasets/toxic_comments.csv')
        print("Файл успешно загружен с платформы")
        return df
    except FileNotFoundError:
        print("Файл не найден на платформе")
    except Exception as e:
        print(f"Ошибка при загрузке файла с платформы: {str(e)}")

    # Попытка загрузить с Google Drive (Colab)
    if is_running_in_colab():
        try:
            from google.colab import drive
            drive.mount('/content/drive')
            drive_path = '/content/drive/MyDrive/Colab Notebooks/toxic_comments.csv'
            df = pd.read_csv(drive_path)
            print("Файл успешно загружен с Google Drive")
            return df
        except Exception as e:
            print(f"Ошибка при загрузке файла с Google Drive: {str(e)}")

    # Если все попытки не удались
    raise Exception("Не удалось загрузить файл ни из одного источника")

In [ ]:
# Функция для очистки текста
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    return text

In [ ]:
# Функция для создания дополнительный признаков
def add_features(df):
    df['text_length'] = df['text'].str.len()
    df['word_count'] = df['text'].str.split().str.len()
    df['unique_words'] = df['text'].apply(lambda x: len(set(str(x).split())))
    df['caps_count'] = df['text'].apply(lambda x: sum(1 for c in str(x) if c.isupper()))
    df['special_char_count'] = df['text'].apply(lambda x: len(re.findall('[^A-Za-z0-9 ]', str(x))))
    return df

In [ ]:
# Функция для удаления стоп-слов
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)


<div class="alert alert-block alert-success">
<b>Успех:</b> Импорты и функции помощники на месте
</div>


Создадим константы:

In [ ]:
MAX_LEN = 128 # Для токенизатора
RANDOM_STATE = 42 # Для перемешивания выборок
TEST_SIZE = 0.25 # Для разделения выборок

### Подготовка данных

Загрузим данные:

In [ ]:
try:
    df = load_dataframe()
except Exception as e:
    print(f"Итоговая ошибка: {str(e)}")

In [ ]:
df.head()

In [ ]:
df.info()

Данные загружены, типы столбцов определились верно, пропусков нет.

#### Датасет для обучения

Датасет слишком большой. Поэтому выделим из него выборку. Сперва посмотрим на баланс классов в общей датафрейме:

In [ ]:
plt.pie(df['toxic'].value_counts(), labels=df['toxic'].value_counts().index.tolist(), colors=sns.color_palette('bright'), autopct='%.0f%%')
plt.title(f'Круговая диаграмма для признака toxic во всём датафрейме')
plt.show()

Соотношение классов в целом столбце: 90/10. Выделим выборку из датафрейма, сохранив соотношение.

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)

for _, sample_idx in split.split(df, df['toxic']):
    sample_df = df.iloc[sample_idx]

In [ ]:
sample_df.info()

Очистим данные от лишних символов:

In [ ]:
sample_df['text'] = sample_df['text'].apply(clean_text)

Создадим новые признаки:

In [ ]:
sample_df = add_features(sample_df)

In [ ]:
sample_df.info()

In [ ]:
plt.pie(sample_df['toxic'].value_counts(), labels=sample_df['toxic'].value_counts().index.tolist(), colors=sns.color_palette('bright'), autopct='%.0f%%')
plt.title(f'Круговая диаграмма для признака toxic')
plt.show()

Признаки созданы, баланс классов сохранен.

Построим теперь облако слов, чтобы оценить, какие слова чаще всего встречаются.

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
sample_df['text_graph'] = sample_df['text'].apply(remove_stopwords)

In [ ]:
text = ' '.join(sample_df['text_graph'].astype(str))

In [ ]:
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

In [ ]:
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Облако слов в датафрейме sample_df')
plt.show()

Датасет для обучения подготовлен. Как видно, из облака слов, наиболее частовстречающиеся слова - article, one, think, wikipedia.

#### Отложенные датасет

Создадим отложенный датасет для оценки качества модели

In [ ]:
df_valid = df.sample(15000).reset_index(drop=True)

In [ ]:
df_valid['text'] = df_valid['text'].apply(clean_text)

In [ ]:
df_valid = add_features(df_valid)

Соотношение классов в отложенной выборке следующее:

In [ ]:
plt.pie(df_valid['toxic'].value_counts(), labels=sample_df['toxic'].value_counts().index.tolist(), colors=sns.color_palette('bright'), autopct='%.0f%%')
plt.title(f'Круговая диаграмма для признака toxic')
plt.show()

### Получение эмбеддингов

Для работы будем использовать модель Bert.

In [ ]:
# Установка устройства (GPU если доступно)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Инициализация токенизатора и модели
tokenizer = BertTokenizer.from_pretrained('unitary/toxic-bert')
model = BertModel.from_pretrained('unitary/toxic-bert')
model = model.to(device)
model.eval()

Установим размер батча.

In [ ]:
batch_size = 128

Проведем токенизацию

In [ ]:
# Токенизация
encoded = tokenizer(
    sample_df['text'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=MAX_LEN,
    return_tensors='pt'
)


<div class="alert alert-block alert-success">
<b>Успех:</b> Настройки токенизации корректно определены
</div>


Получим эмбреддинги

In [ ]:
# Получение эмбеддингов
embeddings = []
n_batches = len(sample_df) // batch_size + (1 if len(sample_df) % batch_size != 0 else 0)

for i in notebook.tqdm(range(n_batches)):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(sample_df))

    batch = {
        'input_ids': encoded['input_ids'][start_idx:end_idx].to(device),
        'attention_mask': encoded['attention_mask'][start_idx:end_idx].to(device)
    }

    with torch.no_grad():
        outputs = model(**batch)
        batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.append(batch_embeddings)

Объединим эмбеддинги

In [ ]:
features = np.concatenate(embeddings)
features_df = pd.DataFrame(
    features,
    columns=[f'bert_feature_{i}' for i in range(features.shape[1])]
)

Объединим с исходным датафреймом

In [ ]:
sample_df = pd.concat([sample_df.reset_index(drop=True), features_df], axis=1)

Посмотрим на результат

In [ ]:
sample_df.head()

Эмбреддинги получены и добавлены в датасет.


<div class="alert alert-block alert-success">
<b>Успех:</b> Эмбеддинги успешно получены
</div>


## Обучение

Теперь приступим к обучению. Разделим данные на выборки и сформируем pipeline.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    sample_df.drop(['text', 'Unnamed: 0', 'toxic', 'text_graph'], axis=1),
    sample_df['toxic'],
    test_size = TEST_SIZE,
    random_state = RANDOM_STATE,
    stratify = sample_df['toxic']
)

X_train.shape, X_test.shape

In [ ]:
num_columns = sample_df.drop(['text', 'Unnamed: 0', 'toxic', 'text_graph'], axis=1).columns

In [ ]:
data_preprocessor = ColumnTransformer(
    [
     ('num', MinMaxScaler(), num_columns)
    ],
    remainder='passthrough'
)

In [ ]:
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', LogisticRegression(random_state=RANDOM_STATE))
])

param_grid = [
    # словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE,
            penalty='l2'
        )],
        'models__solver':['liblinear'],
        'models__C': [0.1, 1, 10],
        'models__class_weight': ['balanced', None],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },

    # словарь для модели LGBMClassifier
    {
        'models': [LGBMClassifier(random_state=RANDOM_STATE)],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },
    {
        'models': [CatBoostClassifier(random_state=RANDOM_STATE)],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },
    {
        'models': [XGBClassifier(random_state=RANDOM_STATE)],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    }
]


In [ ]:
%%time
# Ячейка выполняется около часа.
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    pipe_final,
    param_grid,
    cv=cv,
    scoring='f1',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n',grid_search.best_estimator_)
print ('Метрика лучшей модели на кросс-валидационной выборке:', grid_search.best_score_)

# проверка работы модели на тестовой выборке
pred = grid_search.predict(X_test)
print(f'Метрика F1 на тестовой выборке: {f1_score(y_test, pred)}')

Выгрузим из пайплайна наилучшую модель

In [ ]:
model_toxic = grid_search.best_estimator_.named_steps['models']

In [ ]:
preprocessor = grid_search.best_estimator_.named_steps['preprocessor']

## Оценка качества модели

Добавим эмбреддинги в отложенный датафрейм:

In [ ]:
# Токенизация
encoded = tokenizer(
    df_valid['text'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=MAX_LEN,
    return_tensors='pt'
)

In [ ]:
# Получение эмбеддингов
embeddings = []
n_batches = len(df_valid) // batch_size + (1 if len(df_valid) % batch_size != 0 else 0)

for i in notebook.tqdm(range(n_batches)):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(df_valid))

    batch = {
        'input_ids': encoded['input_ids'][start_idx:end_idx].to(device),
        'attention_mask': encoded['attention_mask'][start_idx:end_idx].to(device)
    }

    with torch.no_grad():
        outputs = model(**batch)
        batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.append(batch_embeddings)

In [ ]:
features = np.concatenate(embeddings)
features_df = pd.DataFrame(
    features,
    columns=[f'bert_feature_{i}' for i in range(features.shape[1])]
)

In [ ]:
df_valid = pd.concat([df_valid.reset_index(drop=True), features_df], axis=1)

Отложенный датасет подготовлен, получим предсказания:

In [ ]:
X_valid_preprocessed = preprocessor.transform(df_valid.drop(['text', 'Unnamed: 0', 'toxic'], axis=1))

In [ ]:
y_pred_valid = model_toxic.predict(X_valid_preprocessed)

Оценим качество предсказанного:

In [ ]:
f1_score(df_valid['toxic'], y_pred_valid)

Метрика соответствует требованиям задания.

Метрика получилась ниже, чем на датасете для обучения. Посмотрим на матрицу ошибок.

In [ ]:
cm = confusion_matrix(df_valid['toxic'], y_pred_valid)

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues_r')
plt.title('Матрица ошибок')
plt.xlabel('Предсказанные значения')
plt.ylabel('Правильные значения')
plt.show()

Как видно из матрицы ошибок, модель чаще ошибается с категорией 1, чем с категорией 0. Но в обоих случаях величина ошибки небольшая.

## Выводы

**В рамках работы сделано:**
1. Загружены данные.
2. Датасет разделен на обучающий и отложенный.
3. Получены эмбреддинги с помощью модели Bert.
4. Обучены несколько моделей, такие как LogisticRegression, LGBMClassifier, CatBoostClassifier, XGBClassifier.
5. Выполнена оценка качества наилучшей модели.

**Достигнутые результаты:**
1. Наилучшей моделью стала LogisticRegression с метрикой F1 на обучеющем датасете 0.91.
2. Метрика F1 на отложенном датасете составила 0,91.
3. Чаще всего модель ошибается с категорией 1, ошибочно присваивая ей категорию 0 (нетоксично).

**Рекомендации для улучшения прогноза:**    
Предоставить дополнительную информацию о твитах, такую как дата твита, время, userid для распределения твитов по пользователям.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны

In [ ]:
#pip freeze > requirements.txt

Содержание файла requirements.txt

In [ ]:
'''
aiobotocore @ file:///C:/b/abs_1c1a_vjay2/croot/aiobotocore_1682537737724/work
aiofiles @ file:///C:/b/abs_9ex6mi6b56/croot/aiofiles_1683773603390/work
aiohttp @ file:///C:/b/abs_b78zt6vo64/croot/aiohttp_1694181126607/work
aioitertools @ file:///tmp/build/80754af9/aioitertools_1607109665762/work
aiosignal @ file:///tmp/build/80754af9/aiosignal_1637843061372/work
aiosqlite @ file:///C:/b/abs_9djc_0pyi3/croot/aiosqlite_1683773915844/work
alabaster @ file:///home/ktietz/src/ci/alabaster_1611921544520/work
anaconda-anon-usage @ file:///C:/b/abs_f4tsjyl9va/croot/anaconda-anon-usage_1695310457827/work
anaconda-catalogs @ file:///C:/b/abs_8btyy0o8s8/croot/anaconda-catalogs_1685727315626/work
anaconda-client @ file:///C:/b/abs_80wttmgui4/croot/anaconda-client_1694625288614/work
anaconda-cloud-auth @ file:///C:/b/abs_5cjpnu6wjb/croot/anaconda-cloud-auth_1694462130037/work
anaconda-navigator @ file:///C:/b/abs_ab00e0_u7e/croot/anaconda-navigator_1695238210954/work
anaconda-project @ file:///C:/ci_311/anaconda-project_1676458365912/work
anyio @ file:///C:/ci_311/anyio_1676425491996/work/dist
appdirs==1.4.4
argon2-cffi @ file:///opt/conda/conda-bld/argon2-cffi_1645000214183/work
argon2-cffi-bindings @ file:///C:/ci_311/argon2-cffi-bindings_1676424443321/work
arrow @ file:///C:/ci_311/arrow_1678249767083/work
astroid @ file:///C:/ci_311/astroid_1678740610167/work
astropy @ file:///C:/ci_311_rebuilds/astropy_1678996071858/work
asttokens @ file:///opt/conda/conda-bld/asttokens_1646925590279/work
async-timeout @ file:///C:/ci_311/async-timeout_1676431518331/work
atomicwrites==1.4.0
attrs @ file:///C:/ci_311/attrs_1676422272484/work
Automat @ file:///tmp/build/80754af9/automat_1600298431173/work
autopep8 @ file:///opt/conda/conda-bld/autopep8_1650463822033/work
Babel @ file:///C:/ci_311/babel_1676427169844/work
backcall @ file:///home/ktietz/src/ci/backcall_1611930011877/work
backports.functools-lru-cache @ file:///tmp/build/80754af9/backports.functools_lru_cache_1618170165463/work
backports.tempfile @ file:///home/linux1/recipes/ci/backports.tempfile_1610991236607/work
backports.weakref==1.0.post1
bcrypt @ file:///C:/ci_311/bcrypt_1676435170049/work
beautifulsoup4 @ file:///C:/b/abs_0agyz1wsr4/croot/beautifulsoup4-split_1681493048687/work
binaryornot @ file:///tmp/build/80754af9/binaryornot_1617751525010/work
black @ file:///C:/b/abs_620t6ndje8/croot/black_1680737261963/work
bleach @ file:///opt/conda/conda-bld/bleach_1641577558959/work
bokeh @ file:///C:/b/abs_e5qs_0dl2w/croot/bokeh_1690546119144/work
boltons @ file:///C:/ci_311/boltons_1677729932371/work
botocore @ file:///C:/b/abs_01gwdn34ju/croot/botocore_1682528022942/work
Bottleneck @ file:///C:/ci_311/bottleneck_1676500016583/work
brotlipy==0.7.0
catboost==1.2.7
certifi @ file:///C:/b/abs_36eb5mzhph/croot/certifi_1690232276943/work/certifi
cffi @ file:///C:/ci_311/cffi_1676423759166/work
chardet @ file:///C:/ci_311/chardet_1676436134885/work
charset-normalizer @ file:///tmp/build/80754af9/charset-normalizer_1630003229654/work
click @ file:///C:/ci_311/click_1676433091657/work
cloudpickle @ file:///C:/b/abs_3796yxesic/croot/cloudpickle_1683040098851/work
clyent==1.2.2
colorama @ file:///C:/ci_311/colorama_1676422310965/work
colorcet @ file:///C:/ci_311/colorcet_1676440389947/work
comm @ file:///C:/ci_311/comm_1678376562840/work
conda @ file:///C:/b/abs_3eb7ewgq2c/croot/conda_1694545461647/work
conda-build @ file:///C:/b/abs_8di4gx5nj5/croot/conda-build_1692366837286/work
conda-content-trust @ file:///C:/b/abs_e3bcpyv7sw/croot/conda-content-trust_1693490654398/work
conda-libmamba-solver @ file:///C:/b/abs_016p0csqp7/croot/conda-libmamba-solver_1691418958509/work/src
conda-pack @ file:///tmp/build/80754af9/conda-pack_1611163042455/work
conda-package-handling @ file:///C:/b/abs_b9wp3lr1gn/croot/conda-package-handling_1691008700066/work
conda-repo-cli==1.0.75
conda-token @ file:///Users/paulyim/miniconda3/envs/c3i/conda-bld/conda-token_1662660369760/work
conda-verify==3.4.2
conda_index @ file:///C:/b/abs_50towt3zan/croot/conda-index_1695311135992/work
conda_package_streaming @ file:///C:/b/abs_6c28n38aaj/croot/conda-package-streaming_1690988019210/work
constantly==15.1.0
contourpy @ file:///C:/ci_311/contourpy_1676431756017/work
cookiecutter @ file:///opt/conda/conda-bld/cookiecutter_1649151442564/work
cryptography @ file:///C:/b/abs_f4do8t8jfs/croot/cryptography_1694444424531/work
cssselect==1.1.0
cycler @ file:///tmp/build/80754af9/cycler_1637851556182/work
cytoolz @ file:///C:/ci_311/cytoolz_1676436342770/work
daal4py==2023.1.1
dask @ file:///C:/b/abs_23lvfodys3/croot/dask-core_1686782960052/work
datasets @ file:///C:/b/abs_a3jy4vrfuo/croot/datasets_1684484478038/work
datashader @ file:///C:/b/abs_8323862uxi/croot/datashader_1692372298149/work
datashape==0.5.4
debugpy @ file:///C:/b/abs_c0y1fjipt2/croot/debugpy_1690906864587/work
decorator @ file:///opt/conda/conda-bld/decorator_1643638310831/work
defusedxml @ file:///tmp/build/80754af9/defusedxml_1615228127516/work
diff-match-patch @ file:///Users/ktietz/demo/mc3/conda-bld/diff-match-patch_1630511840874/work
dill @ file:///C:/ci_311/dill_1676433323862/work
distributed @ file:///C:/b/abs_7509xfv227/croot/distributed_1686866088894/work
docstring-to-markdown @ file:///C:/ci_311/docstring-to-markdown_1677742566583/work
docutils @ file:///C:/ci_311/docutils_1676428078664/work
entrypoints @ file:///C:/ci_311/entrypoints_1676423328987/work
et-xmlfile==1.1.0
executing @ file:///opt/conda/conda-bld/executing_1646925071911/work
fastjsonschema @ file:///C:/ci_311/python-fastjsonschema_1679500568724/work
filelock @ file:///C:/ci_311/filelock_1676427284139/work
flake8 @ file:///C:/ci_311/flake8_1678376624746/work
Flask @ file:///C:/ci_311/flask_1676436667658/work
fonttools==4.25.0
frozenlist @ file:///C:/ci_311/frozenlist_1676428131576/work
fsspec @ file:///C:/b/abs_f9arcwflp9/croot/fsspec_1682526006955/work
future @ file:///C:/ci_311_rebuilds/future_1678998246262/work
gensim @ file:///C:/ci_311/gensim_1677743037820/work
glob2 @ file:///home/linux1/recipes/ci/glob2_1610991677669/work
graphviz==0.20.3
greenlet @ file:///C:/ci_311/greenlet_1676436788118/work
h5py @ file:///C:/b/abs_17fav01gwy/croot/h5py_1691589733413/work
HeapDict @ file:///Users/ktietz/demo/mc3/conda-bld/heapdict_1630598515714/work
holoviews @ file:///C:/b/abs_fa7afixkhc/croot/holoviews_1693378101313/work
huggingface-hub @ file:///C:/b/abs_46tn6fu5ho/croot/huggingface_hub_1686693723427/work
hvplot @ file:///C:/b/abs_2b13wifauw/croot/hvplot_1685998632349/work
hyperlink @ file:///tmp/build/80754af9/hyperlink_1610130746837/work
idna @ file:///C:/ci_311/idna_1676424932545/work
imagecodecs @ file:///C:/b/abs_e2g5zbs1q0/croot/imagecodecs_1695065012000/work
imageio @ file:///C:/ci_311/imageio_1678373794394/work
imagesize @ file:///C:/ci_311/imagesize_1676431905616/work
imbalanced-learn @ file:///C:/b/abs_275a0acaq2/croot/imbalanced-learn_1685025644593/work
importlib-metadata @ file:///C:/b/abs_20ndzb2j6v/croot/importlib-metadata_1678997085534/work
incremental @ file:///tmp/build/80754af9/incremental_1636629750599/work
inflection==0.5.1
iniconfig @ file:///home/linux1/recipes/ci/iniconfig_1610983019677/work
intake @ file:///C:/ci_311_rebuilds/intake_1678999914269/work
intervaltree @ file:///Users/ktietz/demo/mc3/conda-bld/intervaltree_1630511889664/work
ipykernel @ file:///C:/b/abs_07rkft_vaz/croot/ipykernel_1691121700587/work
ipython @ file:///C:/b/abs_e5729i179y/croot/ipython_1694181400005/work
ipython-genutils @ file:///tmp/build/80754af9/ipython_genutils_1606773439826/work
ipywidgets @ file:///C:/b/abs_5awapknmz_/croot/ipywidgets_1679394824767/work
isort @ file:///tmp/build/80754af9/isort_1628603791788/work
itemadapter @ file:///tmp/build/80754af9/itemadapter_1626442940632/work
itemloaders @ file:///opt/conda/conda-bld/itemloaders_1646805235997/work
itsdangerous @ file:///tmp/build/80754af9/itsdangerous_1621432558163/work
jaraco.classes @ file:///tmp/build/80754af9/jaraco.classes_1620983179379/work
jedi @ file:///C:/ci_311/jedi_1679427407646/work
jellyfish @ file:///C:/b/abs_50kgvtnrbj/croot/jellyfish_1695193564091/work
Jinja2 @ file:///C:/ci_311/jinja2_1676424968965/work
jinja2-time @ file:///opt/conda/conda-bld/jinja2-time_1649251842261/work
jmespath @ file:///Users/ktietz/demo/mc3/conda-bld/jmespath_1630583964805/work
joblib @ file:///C:/b/abs_1anqjntpan/croot/joblib_1685113317150/work
json5 @ file:///tmp/build/80754af9/json5_1624432770122/work
jsonpatch @ file:///tmp/build/80754af9/jsonpatch_1615747632069/work
jsonpointer==2.1
jsonschema @ file:///C:/b/abs_d40z05b6r1/croot/jsonschema_1678983446576/work
jupyter @ file:///C:/ci_311/jupyter_1678249952587/work
jupyter-console @ file:///C:/b/abs_82xaa6i2y4/croot/jupyter_console_1680000189372/work
jupyter-events @ file:///C:/b/abs_4cak_28ewz/croot/jupyter_events_1684268050893/work
jupyter-server @ file:///C:/ci_311/jupyter_server_1678228762759/work
jupyter-ydoc @ file:///C:/b/abs_e7m6nh5lao/croot/jupyter_ydoc_1683747253535/work
jupyter_client @ file:///C:/ci_311/jupyter_client_1676424009414/work
jupyter_core @ file:///C:/b/abs_9d0ttho3bs/croot/jupyter_core_1679906581955/work
jupyter_server_fileid @ file:///C:/b/abs_f1yjnmiq_6/croot/jupyter_server_fileid_1684273602142/work
jupyter_server_ydoc @ file:///C:/b/abs_8ai39bligw/croot/jupyter_server_ydoc_1686767445888/work
jupyterlab @ file:///C:/b/abs_c1msr8zz3y/croot/jupyterlab_1686179674844/work
jupyterlab-pygments @ file:///tmp/build/80754af9/jupyterlab_pygments_1601490720602/work
jupyterlab-widgets @ file:///C:/b/abs_38ad427jkz/croot/jupyterlab_widgets_1679055289211/work
jupyterlab_server @ file:///C:/b/abs_e0qqsihjvl/croot/jupyterlab_server_1680792526136/work
kaleido @ file:///C:/b/abs_60smvjz1os/croot/python-kaleido_1689927138239/work
keyring @ file:///C:/b/abs_dbjc7g0dh2/croot/keyring_1678999228878/work
kiwisolver @ file:///C:/ci_311/kiwisolver_1676431979301/work
lazy-object-proxy @ file:///C:/ci_311/lazy-object-proxy_1676432050939/work
lazy_loader @ file:///C:/b/abs_c9jlw06oq1/croot/lazy_loader_1687266162676/work
libarchive-c @ file:///tmp/build/80754af9/python-libarchive-c_1617780486945/work
libmambapy @ file:///C:/b/abs_71g8gec0dd/croot/mamba-split_1694187821755/work/libmambapy
lightgbm==4.5.0
linkify-it-py @ file:///C:/ci_311/linkify-it-py_1676474436187/work
llvmlite @ file:///C:/b/abs_a8i9keuf6p/croot/llvmlite_1683555140340/work
lmdb @ file:///C:/b/abs_556ronuvb2/croot/python-lmdb_1682522366268/work
locket @ file:///C:/ci_311/locket_1676428325082/work
lxml @ file:///C:/b/abs_9e7tpg2vv9/croot/lxml_1695058219431/work
lz4 @ file:///C:/b/abs_064u6aszy3/croot/lz4_1686057967376/work
Markdown @ file:///C:/ci_311/markdown_1676437912393/work
markdown-it-py @ file:///C:/b/abs_a5bfngz6fu/croot/markdown-it-py_1684279915556/work
MarkupSafe @ file:///C:/ci_311/markupsafe_1676424152318/work
matplotlib @ file:///C:/b/abs_085jhivdha/croot/matplotlib-suite_1693812524572/work
matplotlib-inline @ file:///C:/ci_311/matplotlib-inline_1676425798036/work
mccabe @ file:///opt/conda/conda-bld/mccabe_1644221741721/work
mdit-py-plugins @ file:///C:/ci_311/mdit-py-plugins_1676481827414/work
mdurl @ file:///C:/ci_311/mdurl_1676442676678/work
menuinst @ file:///C:/ci_311/menuinst_1678730372782/work
mistune @ file:///C:/ci_311/mistune_1676425149302/work
mkl-fft @ file:///C:/b/abs_19i1y8ykas/croot/mkl_fft_1695058226480/work
mkl-random @ file:///C:/b/abs_edwkj1_o69/croot/mkl_random_1695059866750/work
mkl-service==2.4.0
more-itertools @ file:///tmp/build/80754af9/more-itertools_1637733554872/work
mpmath @ file:///C:/b/abs_7833jrbiox/croot/mpmath_1690848321154/work
msgpack @ file:///C:/ci_311/msgpack-python_1676427482892/work
multidict @ file:///C:/ci_311/multidict_1676428396308/work
multipledispatch @ file:///C:/ci_311/multipledispatch_1676442767760/work
multiprocess @ file:///C:/ci_311/multiprocess_1676442808395/work
munkres==1.1.4
mypy-extensions @ file:///C:/b/abs_8f7xiidjya/croot/mypy_extensions_1695131051147/work
navigator-updater @ file:///C:/b/abs_895otdwmo9/croot/navigator-updater_1695210220239/work
nbclassic @ file:///C:/b/abs_c8_rs7b3zw/croot/nbclassic_1681756186106/work
nbclient @ file:///C:/ci_311/nbclient_1676425195918/work
nbconvert @ file:///C:/ci_311/nbconvert_1676425836196/work
nbformat @ file:///C:/b/abs_5a2nea1iu2/croot/nbformat_1694616866197/work
nest-asyncio @ file:///C:/ci_311/nest-asyncio_1676423519896/work
networkx @ file:///C:/b/abs_e6gi1go5op/croot/networkx_1690562046966/work
nltk @ file:///C:/b/abs_a638z6l1z0/croot/nltk_1688114186909/work
notebook @ file:///C:/b/abs_e2qn6c85jb/croot/notebook_1690985290943/work
notebook_shim @ file:///C:/ci_311/notebook-shim_1678144850856/work
numba @ file:///C:/b/abs_00f2z7znbq/croot/numba_1690878309825/work
numexpr @ file:///C:/b/abs_afm0oewmmt/croot/numexpr_1683221839116/work
numpy @ file:///C:/Users/dev-admin/mkl/numpy_and_numpy_base_1682982345978/work
numpydoc @ file:///C:/ci_311/numpydoc_1676453412027/work
openpyxl==3.0.10
packaging @ file:///C:/b/abs_28t5mcoltc/croot/packaging_1693575224052/work
pandas @ file:///C:/miniconda3/conda-bld/pandas_1692298018988/work
pandocfilters @ file:///opt/conda/conda-bld/pandocfilters_1643405455980/work
panel @ file:///C:/b/abs_a4rd7zrkc6/croot/panel_1695145945642/work
param @ file:///C:/b/abs_f5xzp6ism6/croot/param_1684915326009/work
paramiko @ file:///opt/conda/conda-bld/paramiko_1640109032755/work
parsel @ file:///C:/ci_311/parsel_1676443327188/work
parso @ file:///opt/conda/conda-bld/parso_1641458642106/work
partd @ file:///C:/b/abs_4e2m_ds81n/croot/partd_1693937921136/work
pathlib @ file:///Users/ktietz/demo/mc3/conda-bld/pathlib_1629713961906/work
pathspec @ file:///C:/ci_311/pathspec_1679427644142/work
patsy==0.5.3
pep8==1.7.1
pexpect @ file:///tmp/build/80754af9/pexpect_1605563209008/work
phik==0.12.4
pickleshare @ file:///tmp/build/80754af9/pickleshare_1606932040724/work
Pillow @ file:///C:/b/abs_153xikw91n/croot/pillow_1695134603563/work
pkce @ file:///C:/b/abs_d0z4444tb0/croot/pkce_1690384879799/work
pkginfo @ file:///C:/b/abs_d18srtr68x/croot/pkginfo_1679431192239/work
platformdirs @ file:///C:/b/abs_b6z_yqw_ii/croot/platformdirs_1692205479426/work
plotly @ file:///C:/ci_311/plotly_1676443558683/work
pluggy @ file:///C:/ci_311/pluggy_1676422178143/work
ply==3.11
poyo @ file:///tmp/build/80754af9/poyo_1617751526755/work
prometheus-client @ file:///C:/ci_311/prometheus_client_1679591942558/work
prompt-toolkit @ file:///C:/ci_311/prompt-toolkit_1676425940920/work
Protego @ file:///tmp/build/80754af9/protego_1598657180827/work
psutil @ file:///C:/ci_311_rebuilds/psutil_1679005906571/work
ptyprocess @ file:///tmp/build/80754af9/ptyprocess_1609355006118/work/dist/ptyprocess-0.7.0-py2.py3-none-any.whl
pure-eval @ file:///opt/conda/conda-bld/pure_eval_1646925070566/work
py-cpuinfo @ file:///Users/ktietz/demo/mc3/conda-bld/py-cpuinfo_1629480366017/work
pyarrow==11.0.0
pyasn1 @ file:///Users/ktietz/demo/mc3/conda-bld/pyasn1_1629708007385/work
pyasn1-modules==0.2.8
pycodestyle @ file:///C:/ci_311/pycodestyle_1678376707834/work
pycosat @ file:///C:/ci_311/pycosat_1676438455539/work
pycparser @ file:///tmp/build/80754af9/pycparser_1636541352034/work
pyct @ file:///C:/ci_311/pyct_1676438538057/work
pycurl==7.45.2
pydantic @ file:///C:/b/abs_4bum6ajkso/croot/pydantic_1686125732638/work
PyDispatcher==2.0.5
pydocstyle @ file:///C:/ci_311/pydocstyle_1678402028085/work
pyerfa @ file:///C:/ci_311/pyerfa_1676503994641/work
pyflakes @ file:///C:/ci_311/pyflakes_1678402101687/work
Pygments @ file:///C:/b/abs_fay9dpq4n_/croot/pygments_1684279990574/work
PyJWT @ file:///C:/ci_311/pyjwt_1676438890509/work
pylint @ file:///C:/ci_311/pylint_1678740302984/work
pylint-venv @ file:///C:/ci_311/pylint-venv_1678402170638/work
pyls-spyder==0.4.0
pymystem3==0.2.0
PyNaCl @ file:///C:/ci_311/pynacl_1676445861112/work
pyodbc @ file:///C:/ci_311/pyodbc_1676489976744/work
pyOpenSSL @ file:///C:/b/abs_08f38zyck4/croot/pyopenssl_1690225407403/work
pyparsing @ file:///C:/ci_311/pyparsing_1678502182533/work
PyQt5==5.15.7
PyQt5-sip @ file:///C:/ci_311/pyqt-split_1676428895938/work/pyqt_sip
PyQtWebEngine==5.15.4
pyrsistent @ file:///C:/ci_311/pyrsistent_1676422695500/work
PySocks @ file:///C:/ci_311/pysocks_1676425991111/work
pytest @ file:///C:/b/abs_48heoo_k8y/croot/pytest_1690475385915/work
python-dateutil @ file:///tmp/build/80754af9/python-dateutil_1626374649649/work
python-dotenv @ file:///C:/ci_311/python-dotenv_1676455170580/work
python-json-logger @ file:///C:/b/abs_cblnsm6puj/croot/python-json-logger_1683824130469/work
python-lsp-black @ file:///C:/ci_311/python-lsp-black_1678721855627/work
python-lsp-jsonrpc==1.0.0
python-lsp-server @ file:///C:/b/abs_catecj7fv1/croot/python-lsp-server_1681930405912/work
python-slugify @ file:///tmp/build/80754af9/python-slugify_1620405669636/work
python-snappy @ file:///C:/ci_311/python-snappy_1676446060182/work
pytoolconfig @ file:///C:/ci_311/pytoolconfig_1678402262175/work
pytz @ file:///C:/b/abs_19q3ljkez4/croot/pytz_1695131651401/work
pyviz-comms @ file:///C:/b/abs_6cq38vhwa5/croot/pyviz_comms_1685030740344/work
PyWavelets @ file:///C:/ci_311/pywavelets_1676504105729/work
pywin32==305.1
pywin32-ctypes @ file:///C:/ci_311/pywin32-ctypes_1676427747089/work
pywinpty @ file:///C:/ci_311/pywinpty_1677707791185/work/target/wheels/pywinpty-2.0.10-cp311-none-win_amd64.whl
PyYAML @ file:///C:/ci_311/pyyaml_1676432488822/work
pyzmq @ file:///C:/ci_311/pyzmq_1676423601304/work
QDarkStyle @ file:///tmp/build/80754af9/qdarkstyle_1617386714626/work
qstylizer @ file:///C:/ci_311/qstylizer_1678502012152/work/dist/qstylizer-0.2.2-py2.py3-none-any.whl
QtAwesome @ file:///C:/ci_311/qtawesome_1678402331535/work
qtconsole @ file:///C:/b/abs_eb4u9jg07y/croot/qtconsole_1681402843494/work
QtPy @ file:///C:/ci_311/qtpy_1676432558504/work
queuelib==1.5.0
regex @ file:///C:/ci_311_rebuilds/regex_1679006156792/work
requests @ file:///C:/b/abs_316c2inijk/croot/requests_1690400295842/work
requests-file @ file:///Users/ktietz/demo/mc3/conda-bld/requests-file_1629455781986/work
requests-toolbelt @ file:///C:/b/abs_2fsmts66wp/croot/requests-toolbelt_1690874051210/work
responses @ file:///tmp/build/80754af9/responses_1619800270522/work
rfc3339-validator @ file:///C:/b/abs_ddfmseb_vm/croot/rfc3339-validator_1683077054906/work
rfc3986-validator @ file:///C:/b/abs_6e9azihr8o/croot/rfc3986-validator_1683059049737/work
rope @ file:///C:/ci_311/rope_1678402524346/work
Rtree @ file:///C:/ci_311/rtree_1676455758391/work
ruamel-yaml-conda @ file:///C:/ci_311/ruamel_yaml_1676455799258/work
ruamel.yaml @ file:///C:/ci_311/ruamel.yaml_1676439214109/work
s3fs @ file:///C:/b/abs_adfhcfx438/croot/s3fs_1682551489845/work
safetensors @ file:///C:/b/abs_a8nnkwknpv/croot/safetensors_1692949064285/work
scikit-image @ file:///C:/b/abs_2075zg1pia/croot/scikit-image_1682528361447/work
scikit-learn @ file:///C:/b/abs_55olq_4gzc/croot/scikit-learn_1690978955123/work
scikit-learn-intelex==20230426.121932
scipy==1.11.1
Scrapy @ file:///C:/ci_311/scrapy_1678502587780/work
seaborn @ file:///C:/ci_311/seaborn_1676446547861/work
Send2Trash @ file:///tmp/build/80754af9/send2trash_1632406701022/work
service-identity @ file:///Users/ktietz/demo/mc3/conda-bld/service_identity_1629460757137/work
shap==0.46.0
sip @ file:///C:/ci_311/sip_1676427825172/work
six @ file:///tmp/build/80754af9/six_1644875935023/work
slicer==0.0.8
smart-open @ file:///C:/ci_311/smart_open_1676439339434/work
sniffio @ file:///C:/ci_311/sniffio_1676425339093/work
snowballstemmer @ file:///tmp/build/80754af9/snowballstemmer_1637937080595/work
sortedcontainers @ file:///tmp/build/80754af9/sortedcontainers_1623949099177/work
soupsieve @ file:///C:/b/abs_a989exj3q6/croot/soupsieve_1680518492466/work
Sphinx @ file:///C:/ci_311/sphinx_1676434546244/work
sphinxcontrib-applehelp @ file:///home/ktietz/src/ci/sphinxcontrib-applehelp_1611920841464/work
sphinxcontrib-devhelp @ file:///home/ktietz/src/ci/sphinxcontrib-devhelp_1611920923094/work
sphinxcontrib-htmlhelp @ file:///tmp/build/80754af9/sphinxcontrib-htmlhelp_1623945626792/work
sphinxcontrib-jsmath @ file:///home/ktietz/src/ci/sphinxcontrib-jsmath_1611920942228/work
sphinxcontrib-qthelp @ file:///home/ktietz/src/ci/sphinxcontrib-qthelp_1611921055322/work
sphinxcontrib-serializinghtml @ file:///tmp/build/80754af9/sphinxcontrib-serializinghtml_1624451540180/work
spyder @ file:///C:/b/abs_e99kl7d8t0/croot/spyder_1681934304813/work
spyder-kernels @ file:///C:/b/abs_e788a8_4y9/croot/spyder-kernels_1691599588437/work
SQLAlchemy @ file:///C:/ci_311/sqlalchemy_1676446707912/work
stack-data @ file:///opt/conda/conda-bld/stack_data_1646927590127/work
statsmodels @ file:///C:/b/abs_7bth810rna/croot/statsmodels_1689937298619/work
sympy==1.13.1
tables @ file:///C:/b/abs_0626auep9v/croot/pytables_1691623892917/work
tabulate @ file:///C:/ci_311/tabulate_1676494503192/work
TBB==0.2
tblib @ file:///Users/ktietz/demo/mc3/conda-bld/tblib_1629402031467/work
tenacity @ file:///C:/b/abs_ddkoa9nju6/croot/tenacity_1682972298929/work
terminado @ file:///C:/ci_311/terminado_1678228513830/work
text-unidecode @ file:///Users/ktietz/demo/mc3/conda-bld/text-unidecode_1629401354553/work
textdistance @ file:///tmp/build/80754af9/textdistance_1612461398012/work
threadpoolctl @ file:///Users/ktietz/demo/mc3/conda-bld/threadpoolctl_1629802263681/work
three-merge @ file:///tmp/build/80754af9/three-merge_1607553261110/work
tifffile @ file:///C:/b/abs_45o5chuqwt/croot/tifffile_1695107511025/work
tinycss2 @ file:///C:/ci_311/tinycss2_1676425376744/work
tldextract @ file:///opt/conda/conda-bld/tldextract_1646638314385/work
tokenizers @ file:///C:/b/abs_e34f9vyhry/croot/tokenizers_1687191954304/work
toml @ file:///tmp/build/80754af9/toml_1616166611790/work
tomlkit @ file:///C:/ci_311/tomlkit_1676425418821/work
toolz @ file:///C:/ci_311/toolz_1676431406517/work
torch==2.5.1
tornado @ file:///C:/b/abs_61jhmrrua1/croot/tornado_1690848767317/work
tqdm @ file:///C:/b/abs_f76j9hg7pv/croot/tqdm_1679561871187/work
traitlets @ file:///C:/ci_311/traitlets_1676423290727/work
transformers @ file:///C:/b/abs_375hgbdwge/croot/transformers_1693308365188/work
Twisted @ file:///C:/b/abs_f1pc_rieoy/croot/twisted_1683796899561/work
twisted-iocpsupport @ file:///C:/ci_311/twisted-iocpsupport_1676447612160/work
typing_extensions==4.12.2
tzdata @ file:///croot/python-tzdata_1690578112552/work
uc-micro-py @ file:///C:/ci_311/uc-micro-py_1676457695423/work
ujson @ file:///C:/ci_311/ujson_1676434714224/work
Unidecode @ file:///tmp/build/80754af9/unidecode_1614712377438/work
urllib3 @ file:///C:/b/abs_889_loyqv4/croot/urllib3_1686163174463/work
w3lib @ file:///Users/ktietz/demo/mc3/conda-bld/w3lib_1629359764703/work
watchdog @ file:///C:/ci_311/watchdog_1676457923624/work
wcwidth @ file:///Users/ktietz/demo/mc3/conda-bld/wcwidth_1629357192024/work
webencodings==0.5.1
websocket-client @ file:///C:/ci_311/websocket-client_1676426063281/work
Werkzeug @ file:///C:/b/abs_8578rs2ra_/croot/werkzeug_1679489759009/work
whatthepatch @ file:///C:/ci_311/whatthepatch_1678402578113/work
widgetsnbextension @ file:///C:/b/abs_882k4_4kdf/croot/widgetsnbextension_1679313880295/work
win-inet-pton @ file:///C:/ci_311/win_inet_pton_1676425458225/work
wrapt @ file:///C:/ci_311/wrapt_1676432805090/work
xarray @ file:///C:/b/abs_5bkjiynp4e/croot/xarray_1689041498548/work
xgboost==2.1.3
xlwings @ file:///C:/ci_311_rebuilds/xlwings_1679013429160/work
xxhash @ file:///C:/ci_311/python-xxhash_1676446168786/work
xyzservices @ file:///C:/ci_311/xyzservices_1676434829315/work
y-py @ file:///C:/b/abs_b7f5go6r0j/croot/y-py_1683662173571/work
yapf @ file:///tmp/build/80754af9/yapf_1615749224965/work
yarl @ file:///C:/ci_311/yarl_1676432870023/work
ypy-websocket @ file:///C:/b/abs_4e65ywlnv8/croot/ypy-websocket_1684172103529/work
zict @ file:///C:/b/abs_fc7elavmem/croot/zict_1682698759288/work
zipp @ file:///C:/ci_311/zipp_1676426100491/work
zope.interface @ file:///C:/ci_311/zope.interface_1676439868776/work
zstandard==0.19.0
'''